## Install

In [0]:
!pip3 install torch torchvision numpy

## Imports

In [0]:
from matplotlib import pyplot as plt
import numpy as np

import torch as th
from torch import nn
import torchvision
from torchvision import transforms

## Config

In [0]:
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cuda


## MNIST Dataset

In [0]:
# Download and construct MNIST dataset.
train_dataset = torchvision.datasets.MNIST(root='~/code/data/mnist/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='~/code/data/mnist/',
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True)

# Data loader (input pipeline)
train_loader = th.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)
test_loader = th.utils.data.DataLoader(dataset=test_dataset,
                                       batch_size=batch_size,
                                       shuffle=False)

## Model

In [0]:
# Convolutional neural network (with 2 convolutional layers).
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.fc = nn.Linear(7 * 7 * 32, num_classes)
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer.
loss_fn = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [0]:
num_steps = len(train_loader)
for epoch in range(num_epochs):
  for step, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (step + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{num_steps}], '
            f'Loss: {loss.item():.4}')

Epoch [1/5], Step [100/600], Loss: 0.1623
Epoch [1/5], Step [200/600], Loss: 0.09485
Epoch [1/5], Step [300/600], Loss: 0.07614
Epoch [1/5], Step [400/600], Loss: 0.06426
Epoch [1/5], Step [500/600], Loss: 0.05154
Epoch [1/5], Step [600/600], Loss: 0.07569
Epoch [2/5], Step [100/600], Loss: 0.05286
Epoch [2/5], Step [200/600], Loss: 0.07079
Epoch [2/5], Step [300/600], Loss: 0.03439
Epoch [2/5], Step [400/600], Loss: 0.03838
Epoch [2/5], Step [500/600], Loss: 0.029
Epoch [2/5], Step [600/600], Loss: 0.05402
Epoch [3/5], Step [100/600], Loss: 0.05124
Epoch [3/5], Step [200/600], Loss: 0.07254
Epoch [3/5], Step [300/600], Loss: 0.04126
Epoch [3/5], Step [400/600], Loss: 0.04214
Epoch [3/5], Step [500/600], Loss: 0.006395
Epoch [3/5], Step [600/600], Loss: 0.01886
Epoch [4/5], Step [100/600], Loss: 0.04819
Epoch [4/5], Step [200/600], Loss: 0.01187
Epoch [4/5], Step [300/600], Loss: 0.01625
Epoch [4/5], Step [400/600], Loss: 0.02155
Epoch [4/5], Step [500/600], Loss: 0.003221
Epoch [4/5],

## Test

In [0]:
with th.no_grad():
  correct, total = 0, 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = th.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  accuracy = correct / total
  print(f'Accuracy of model on 10000 test images: {100 * accuracy:0.2f}%')

Accuracy of model on 10000 test images: 99.22%


## Save model

In [0]:
th.save(model.state_dict(), '/tmp/model.ckpt')